## Travailler avec un Notebook dans le village

J'expérimente ici pour voir comment il est possible de travailler avec une base qui ne contient que notre "village", faire des requêtes dedans mais sans toucher au contenu de la base.

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from batid.models import Building

Création d'un décorateur @in_village qui permet de faire des requêtes dans le village de test uniquement.

In [3]:
from contextlib import ContextDecorator
from django.db import transaction
from django.db import connection
from batid.tests.test_search import create_village

# création d'un décorateur qui vide la table building (et les tables associées en cascade)
class insert_village(ContextDecorator):
    cursor = connection.cursor()
    def __enter__(self):
        self.cursor.execute('BEGIN TRANSACTION')
        self.cursor.execute('TRUNCATE TABLE batid_building CASCADE')
        create_village()
        return self

    def __exit__(self, *exc):
        self.cursor.execute('ROLLBACK TRANSACTION')
        return False

# le décorateur précédent est appelé par le décorateur transaction.atomic() qui permet de ne pas faire de commit 
# dans la base après chaque execution SQL, mais de tout wrapper dans une transaction que l'on peut rollback 
# une fois terminée
def in_village(func):
    from django.db import transaction
    return transaction.atomic(insert_village()(func))

### Obtenir le geojson du village avec les adresses des batiments associés dans les propriétés du geojson

In [35]:
from django.db import transaction
import json

@in_village
def village_geojson():
    cursor = connection.cursor()
    q = """SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(ST_AsGeoJSON(limited_building.*)::json)
) 
FROM (
    SELECT ST_Transform(b.shape, 4326), b.*, array_agg(json_build_object('address_id', ba.id, 'address', CONCAT(ba.street_number, ' ', ba.street_rep, ' ', ba.street_type, ' ',  ba.street_name, ', ',  ba.city_name))) as add_id FROM batid_building as b
    join batid_building_addresses bba on bba.building_id = b.id
    join batid_address ba on bba.address_id = ba.id
    group by b.id 
) as limited_building;"""
    
    cursor.execute(q)
    for row in cursor.fetchall():
        print(json.dumps(row[0]))
    #return buildings

In [36]:
geojson = village_geojson()
geojson

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "MultiPolygon", "coordinates": [[[[5.636205093, 45.069951257], [5.636231215, 45.069992981], [5.636315155, 45.06996667], [5.636245255, 45.06983229], [5.636088041, 45.069868443], [5.636144943, 45.069969798], [5.636205093, 45.069951257]]]]}, "properties": {"id": 45610, "rnb_id": "E6TGPMY2TV35", "source": "bdtopo", "point": {"type": "Point", "crs": {"type": "name", "properties": {"name": "EPSG:2154"}}, "coordinates": [907432.9866169977, 6444646.25]}, "ext_bdnb_id": "", "ext_bdtopo_id": "BATIMENT0000000302581487", "shape": {"type": "MultiPolygon", "crs": {"type": "name", "properties": {"name": "EPSG:2154"}}, "coordinates": [[[[907433.4, 6444651], [907435.3, 6444655.7], [907442, 6444653], [907437, 6444637.9], [907424.5, 6444641.5], [907428.6, 6444652.9], [907433.4, 6444651]]]]}, "created_at": "2023-10-02T10:38:46.687+00:00", "updated_at": "2023-10-02T10:38:46.687+00:00", "add_id": [{"address_id": "IMB/38436/X